In [1]:
import numpy as np
import math as m
import copy
import itertools
import time

In [2]:
def create_grid(txt):
    #This assumes a square grid
    grid_len = len(txt.replace(' ','').split('\n')[1])
    lines = txt.replace(' ','').replace('\n','')
    digits = list(map(int,lines))
    grid = np.array(digits).reshape(grid_len,grid_len)
    return grid

In [3]:
def create_candidates(grid):
    row_col_len = len(grid[0])
    ret_candidates = [np.array([(set(range(1,10)) - (set(grid[row]) | set(grid[:,col]) | set([grid[row - row % 3 + i][col - col % 3 + j] for i in range(3) for j in range(3) if row-row%3+i<row_col_len if col-col%3+j<row_col_len]))) if not grid[row][col] else set([]) for col in range(row_col_len)]) for row in range(row_col_len)]
    return np.array(ret_candidates)

In [4]:
def reduce_candidates(candidates,row,col,num):
    #When we fill a cell we know the value of, with value num at (row,col)
    #row,col,box candidates all must have that number removed from their set
    adjusted_cells = []
    for i in range(9):
        if candidates[row][i] & set([num]):
            #If num is in the candidates in this cell
            candidates[row][i] -= set([num])
            adjusted_cells.append((row,i))
        if candidates[i][col] & set([num]):
            candidates[i][col] -= set([num])
            adjusted_cells.append((i,col))
            
    for i in range(3):
        for j in range(3):
            if candidates[row - row % 3 + i][col - col % 3 + j] & set([num]):
                candidates[row - row % 3 + i][col - col % 3 + j] -= set([num])
                adjusted_cells.append((row - row % 3 + i,col - col % 3 + j))
    return list(set(adjusted_cells))

In [5]:
def naked_singles(grid,candidates):
    filled = False
    for i in range(9):
        for j in range(9):
            if len(candidates[i][j]) == 1:
                single = list(candidates[i][j])[0]
                grid[i][j] = single
                #print("Placing {0} in row {1} col {2}".format(single,i,j))
                candidates[i][j] = set([])
                #But any candidate adjustments could have created more singles than we thought previously, so go back through again
                reduce_candidates(candidates,i,j,single)
                filled = True
    return filled

In [6]:
def hidden_singles_up(grid,candidates):
    #Check for each cell whether or not a candidate value in that cell
    #is the only spot in row/col/box with that value, then it MUST be that value
    ret = False
    for row in range(9):
        indices = [i for i in range(9) if candidates[row][i]]
        for i in range(1,len(indices)//2 + len(indices)%2 + 1):
            for index_comb in itertools.combinations(indices,i):
                other_cells = list(set(indices) - set(list(index_comb)))
                comb_candidates = set().union(*[candidates[row][i] for i in index_comb])
                other_candidates = set().union(*[candidates[row][i] for i in other_cells])
                hiddens = comb_candidates - other_candidates
                if len(hiddens) == i:
                    #In each of these cells in index_comb, remove candidates not in hiddens
                    for cell in index_comb:
                        if (candidates[row][cell] - hiddens):
                            candidates[row][cell] = candidates[row][cell] & hiddens
                            ret = True
                    if ret:
                        #print("Row",row,hiddens)
                        return ret
                    
    for col in range(9):
        indices = [i for i in range(9) if candidates[i][col]]
        for i in range(1,len(indices)//2 + len(indices)%2 + 1):
            for index_comb in itertools.combinations(indices,i):
                other_cells = list(set(indices) - set(list(index_comb)))
                comb_candidates = set().union(*[candidates[i][col] for i in index_comb])
                other_candidates = set().union(*[candidates[i][col] for i in other_cells])
                hiddens = comb_candidates - other_candidates
                if len(hiddens) == i:
                    #In each of these cells in index_comb, remove candidates not in hiddens
                    for cell in index_comb:
                        if (candidates[cell][col] - hiddens):
                            candidates[cell][col] = candidates[cell][col] & hiddens
                            ret = True
                    if ret:
                        #print("Col",col,hiddens)
                        return ret
                    
    for box in range(9):
        indices = [index for index in range(9) if candidates[3*(box//3)+index//3][3*(box%3)+index%3]]
        for i in range(1,len(indices)//2 + len(indices)%2 + 1):
            for index_comb in itertools.combinations(indices,i):
                other_cells = list(set(indices) - set(list(index_comb)))
                
                comb_candidates = set([])
                for index in index_comb:
                    comb_candidates = comb_candidates | candidates[3*(box//3)+index//3][3*(box%3)+index%3]
                
                other_candidates = set([])
                for index in other_cells:
                    other_candidates = other_candidates | candidates[3*(box//3)+index//3][3*(box%3)+index%3]
                    
                hiddens = comb_candidates - other_candidates
                if len(hiddens) == i:
                    #In each of these cells in index_comb, remove candidates not in hiddens
                    for ind in index_comb:
                        if (candidates[3*(box//3)+ind//3][3*(box%3)+ind%3] - hiddens):
                            candidates[3*(box//3)+ind//3][3*(box%3)+ind%3] = candidates[3*(box//3)+ind//3][3*(box%3)+ind%3] & hiddens
                            ret = True
                    if ret:
                        #print("Box",box,hiddens,index_comb)
                        return ret
                    
    return ret

In [7]:
def naked_pair_up(grid,candidates):
    ret = False
    for row in range(9):
        #vals = [x for x in candidates[row] if x]
        indices = [i for i in range(9) if candidates[row][i]]
        for i in range(2,len(indices)//2 + len(indices)%2 + 1):#if len(indices) > (i + 1):
            #number of unfilled cells = len(indices), pointless to check combination length == number of unfilled
            for index_comb in itertools.combinations(indices,i):
                checkset = set([])
                for index in index_comb:
                    #Determine if the total candidates in these cells is 2
                    checkset = checkset | candidates[row][index]
                if len(checkset) == i:
                    #print("Do something row",row)
                    #print("Combination amnt",i,"Combination",index_comb)
                    #print(candidates[row])
                    #For everything not in index_comb, remove values not in checkset
                    for ind in indices:
                        if ind not in index_comb and (checkset & candidates[row][ind]):
                            ret = True
                            candidates[row][ind] -= checkset
                    if ret:
                        #print("row",row,checkset)
                        return ret

    for col in range(9):
        #vals = [x for x in candidates[:,col] if x]
        indices = [i for i in range(9) if candidates[i][col]]
        for i in range(2,len(indices)//2 + len(indices)%2 + 1):
            for index_comb in itertools.combinations(indices,i):
                checkset = set([])
                for index in index_comb:
                    checkset = checkset | candidates[index][col]
                if len(checkset) == i:
                    #print("Do something col",i)
                    #print(col,index_comb)
                    for ind in indices:
                        if ind not in index_comb and (checkset & candidates[ind][col]):
                            ret = True
                            candidates[ind][col] -= checkset
                    if ret:
                        #print("col",col,checkset)
                        return ret

    for box in range(9):
        #candidate_subset = [candidates[3*(box//3)+index//3][3*(box%3)+index%3] for index in range(9) if candidates[3*(box//3)+index//3][3*(box%3)+index%3]]
        indices = [index for index in range(9) if candidates[3*(box//3)+index//3][3*(box%3)+index%3]]
        for i in range(2,len(indices)//2 + len(indices)%2 + 1):
            for index_comb in itertools.combinations(indices,i):
                checkset = set([])
                for index in index_comb:
                    checkset = checkset | candidates[3*(box//3)+index//3][3*(box%3)+index%3]
                if len(checkset) == i:
                    #print("Do something box",i)
                    #print(box,index_comb)
                    for ind in indices:
                        if ind not in index_comb and (checkset & candidates[3*(box//3)+ind//3][3*(box%3)+ind%3]):
                            ret = True
                            candidates[3*(box//3)+ind//3][3*(box%3)+ind%3] -= checkset
                    if ret:
                        #print("box",box,checkset)
                        return ret
    return ret

In [8]:
def pointing_pairs(grid,candidates):
    #When a value in the candidates of a box align in a row or column
    #Then that value may be removed from cells along that alignment outside the box
    ret = False #Just a checker if we did any changes
    for box in range(9):
        #print("Box:",box)
        box_candidates = [[candidates[3*(box//3) + j][3*(box%3) + i] for i in range(3)] for j in range(3)]
        #print("Box candidates:",box_candidates)
        for num in list(set().union(*[item for sublist in box_candidates for item in sublist])):
            #print("Num:",num)
            #Get all the positions in the box with this number, check if aligned in one row/col
            rows = set([row for row in range(3) for col in range(3) if num in box_candidates[row][col]])
            cols = set([col for row in range(3) for col in range(3) if num in box_candidates[row][col]])
            #adjust them according to actual box location
            rows = np.array(list(rows)) + 3*(box//3)
            cols = np.array(list(cols)) + 3*(box%3)
            #print("Rows containing num:",rows)
            #print("Cols containing num:",cols)
            if len(rows) == 1:
                #Then we have num pointing along this row, remove candidates along this row
                for i in range(9):
                    if i not in cols:
                        #print((rows[0],i))
                        #print(candidates[rows[0]][i] - set([num]))
                        if num in candidates[rows[0]][i]:
                            #print("Pointing along row {0} removing {1} based on box {2}".format(rows[0],num,box))
                            ret = True
                            candidates[rows[0]][i] -= set([num])
            elif len(cols) == 1:
                for i in range(9):
                    if i not in rows:
                        #print((i,cols[0]))
                        #print(candidates[i][cols[0]] - set([num]))
                        if num in candidates[i][cols[0]]:
                            #print("Pointing along col {0} removing {1} based on box {2}".format(cols[0],num,box))
                            ret = True
                            #print(i,cols[0])
                            #print(candidates[i][cols[0]])
                            candidates[i][cols[0]] -= set([num])
    return ret

In [9]:
def box_line_reduction(grid,candidates):
    #When a value along a row or column is confined in one box only
    #May remove that value from candidates outside this row/column in that box
    
    #Check and copy for column version?
    ret = False
    for line in range(9):
        for _type in range(2):
            if _type:
                line_candidates = candidates[line]
            else:
                line_candidates = candidates[:,line]
            for num in list(set().union(*line_candidates)):
                other_lines = np.array([i for i in range(9) if num in line_candidates[i]])
                boxes = set(other_lines//3)
                if len(boxes) == 1:
                    for i in range(3*(line//3),3*(line//3)+3):
                        for j in range(3*(other_lines[0]//3),3*(other_lines[0]//3)+3):
                            if i != line:
                                if _type:
                                    if num in candidates[i][j]:
                                        #print("Removing {0} in box from row {1} alignment".format(num,line))
                                        ret = True
                                    candidates[i][j] -= set([num])
                                else:
                                    if num in candidates[j][i]:
                                        #print("Removing {0} in box from col {1} alignment".format(num,line))
                                        ret = True
                                    candidates[j][i] -= set([num])
    return ret

In [10]:
def x_wing(grid,candidates):
    #Take two cells, if they are the only cells with a value in row, and a different row in same cols also only have that value
    #Likewise for col checks across different columns and row aligned
    ret = False
    master_row_lst = []
    master_wing_rows = []
    master_filled_lst = []
    for num in range(1,10):
        for row in range(9):
            #Look along the row
            total_candidates = set().union(*candidates[row])
            if num in total_candidates:
                #Find number of cells with this
                col_lst = []
                filled_lst = []
                for i in range(9):
                    if num in candidates[row][i]:
                        #This cell has this number
                        col_lst.append(i)
                    elif candidates[row][i] == set([]):
                        #Empty candidates, filled this value already but could be used as filler to swordfish/jellyfish
                        filled_lst.append(i)
                if len(col_lst) in [2,3,4]: #x-wing,swordfish,jellyfish; latter 2 can contain incomplete lengths, ensures at least two legitimate unfilled cells with this number along a region
                    #Get combinations of filler cells 
                    for cell_count in range(2,5):
                        if cell_count >= len(col_lst):
                            combs = itertools.combinations(filled_lst,cell_count-len(col_lst)) #combinations of len 0-2
                            #Add combination of filler to legitimate cells with candidates
                            for comb in combs:
                                wing_cols = list(col_lst+list(comb))
                                #print(wing_cols)
                                wing_cols.sort()
                                wing = list([num]+wing_cols)
                                master_row_lst.append(wing)
                                master_wing_rows.append(row)
                    #Now master contains all those combinations, we can check if any wing possibilities line up 2,3,4 total cells and eliminate accordingly
    wing = []
    uniques,counts = np.unique(master_row_lst,return_counts=True)
    #Sorting counts from lowest to highest
    try:
        counts,uniques = (list(t) for t in zip(*sorted(zip(counts,uniques))))
    except:
        pass
    for i in range(len(counts)):
        if counts[i] in [2,3,4]:
            wing = uniques[i]
            
            #Since we have ensured at least two legitimate cells per region, and we check for hidden singles prior to this
            #We definitely have some kind of x-wing

            #Here we look along rows for a given number, so we eliminate in columns
            #Must dictate the safe rows per column to not adjust candidates for
            safe_rows = []
            k = 0
            stop = len(master_wing_rows)
            #Loop through the wing list and grab index where we find the wing we know is x-wing like
            while k <= stop:
                #Attempt to get the index, could throw error indicating there are no more left to find in what's left of the list
                #Then it breaks out of loop
                try:
                    t = master_row_lst.index(wing,k,stop)
                    safe_rows.append(master_wing_rows[t])
                    k = t + 1
                except:
                    break
            #Checking to see if cubic type that we need
            if  counts[i] == len(wing[1:]):
                #Candidate eliminations
                for j in range(9):
                    for col in wing[1:]: #wing[1:] contains all of the columns in this x-wing like seen across multiple rows
                        #Eliminate this number value from these columns where we aren't in the safe_rows of the x-wing
                        if j not in safe_rows:
                            if candidates[j][col] & set([wing[0]]): #wing[0] is the number we are attempting to find x-wing like for to eliminate candidates with
                                ret = True
                                candidates[j][col] -= set([wing[0]])
                if ret:
                    #print("Removing {0} from cols {1} w/ x safe rows {2}!!".format(wing[0],wing[1:],safe_rows))
                    return ret
            else:
                #Not enough match to be an x-wing like
                continue

    master_col_lst = []
    master_wing_cols = []
    master_filled_lst = []
    for num in range(1,10):
        for col in range(9):
            #Look along the row
            total_candidates = set().union(*candidates[:,col])
            if num in total_candidates:
                #Find number of cells with this
                row_lst = []
                filled_lst = []
                for i in range(9):
                    if num in candidates[i][col]:
                        #This cell has this number
                        row_lst.append(i)
                    elif candidates[i][col] == set([]):
                        #Empty candidates, filled this value already but could be used as filler to swordfish/jellyfish
                        filled_lst.append(i)
                if len(row_lst) in [2,3,4]: #x-wing,swordfish,jellyfish; latter 2 can contain incomplete lengths, ensures at least two legitimate unfilled cells with this number along a region
                    #Get combinations of filler cells 
                    for cell_count in range(2,5):
                        if cell_count >= len(row_lst):
                            combs = itertools.combinations(filled_lst,cell_count-len(row_lst)) #combinations of len 0-2
                            #Add combination of filler to legitimate cells with candidates
                            for comb in combs:
                                wing_rows = list(row_lst+list(comb))
                                #print(wing_rows)
                                wing_rows.sort()
                                wing = list([num]+wing_rows)
                                master_col_lst.append(wing)
                                master_wing_cols.append(col)
                    #Now master contains all those combinations, we can check if any wing possibilities line up 2,3,4 total cells and eliminate accordingly
    wing = []
    uniques,counts = np.unique(master_col_lst,return_counts=True)
    #Sorting counts from lowest to highest
    try:
        counts,uniques = (list(t) for t in zip(*sorted(zip(counts,uniques))))
    except:
        pass
    for i in range(len(counts)):
        if counts[i] in [2,3,4]:
            wing = uniques[i]
            
            #Since we have ensured at least two legitimate cells per region, and we check for hidden singles prior to this
            #We definitely have some kind of x-wing

            #Here we look along rows for a given number, so we eliminate in columns
            #Must dictate the safe rows per column to not adjust candidates for
            safe_cols = []
            k = 0
            stop = len(master_wing_cols)
            #Loop through the wing list and grab index where we find the wing we know is x-wing like
            while k <= stop:
                #Attempt to get the index, could throw error indicating there are no more left to find in what's left of the list
                #Then it breaks out of loop
                try:
                    t = master_col_lst.index(wing,k,stop)
                    safe_cols.append(master_wing_cols[t])
                    k = t + 1
                except:
                    break
            
            #Checking to see if cubic type that we need
            if counts[i] == len(wing[1:]):
                #Candidate eliminations
                for j in range(9):
                    for row in wing[1:]: #wing[1:] contains all of the columns in this x-wing like seen across multiple rows
                        #Eliminate this number value from these columns where we aren't in the safe_rows of the x-wing
                        if j not in safe_cols:
                            if candidates[row][j] & set([wing[0]]): #wing[0] is the number we are attempting to find x-wing like for to eliminate candidates with
                                ret = True
                                candidates[row][j] -= set([wing[0]])
                if ret:
                    #print("Removing {0} from cols {1} w/ x safe rows {2}!!".format(wing[0],wing[1:],safe_cols))
                    return ret
            else:
                #Not enough match to be an x-wing like
                continue
    
    return ret

In [11]:
def y_wing(grid,candidates):
    #This is effectively an x-wing,but missing the corner
    #It dictates that the three cells share 3 total candidate values and that the corner that 'sees' the other two cells
    #regardless of value will enforce that the fourth cell that also sees the couple cannot be the tertiary value
    #E.g. ab is seen by two cells bc and ac. Therefore regardless of ab cell value one of couple is c and cell they both see cannot be c
    
    #All three cells cannot be in the same region, otherwise its merely a naked triple
    #Remove all cells with candidate c seen by cells bc and ac, if handling two cells in a box this is more than one intersection
    
    #Basic: Search for cells with two candidates
        #Then search all cells seen by this cell for cells of two candidates
        #For every combination of coupled seen cells by 1st cell, check y-wing possibility
        #Must ensure not all three cells in same region, otherwise just a naked triple
        #and that no two have identical candidates
    ret = False
    for row in range(9):
        for col in range(9):
            if len(candidates[row][col]) == 2:
                #Possible y-wing ab cell
                #Get all the cells that are seen by this cell
                #print("Start")
                #print(row,col)
                possible_cells = []
                for col2 in range(9):
                    if col != col2 and len(candidates[row][col2]) == 2:
                        possible_cells.append((row,col2))
                for row2 in range(9):
                    if row != row2 and len(candidates[row2][col]) == 2:
                        possible_cells.append((row2,col))
                for i in range(3*(row//3),3*(row//3)+3):
                    for j in range(3*(col//3),3*(col//3)+3):
                        if (i,j) != (row,col):
                            if len(candidates[i][j]) == 2:
                                possible_cells.append((i,j))
                #print(possible_cells)
                possible_cells = list(set(possible_cells)) #Just eliminates duplicates that occur from box search
                #print(possible_cells)
                #print("")
                if len(possible_cells) >= 2:
                    #Then we have at least the number of two candidate cells to enable a y-wing
                    #Splice through all two cell subsets of list
                    for comb in itertools.combinations(possible_cells,2):
                        #Comb should be tuple of two location tuples
                        
                        #Check if same region
                        box_cells = [(3*(row//3)+i,3*(col//3)+j) for i in range(3) for j in range(3)]
                        if (comb[0][0] == comb[1][0] == row) or (comb[0][1] == comb[1][1] == col) or (comb[0] in box_cells and comb[1] in box_cells): #(row,col) obviously in the same box
                            #print("Same region")
                            continue
                        else:
                            triple_check = set([])
                            for loc in comb:
                                triple_check = triple_check | candidates[loc[0],loc[1]]
                            triple_check = triple_check | candidates[row][col]
                            if len(triple_check) == 3:
                                #print(row,col)
                                #print(comb)
                                #print(grid)
                                #print(candidates)

                                #Ok now we gotta check if we have ab, bc, ac condition met
                                c_cell1 = candidates[comb[0][0],comb[0][1]] - candidates[row][col]
                                c_cell2 = candidates[comb[1][0],comb[1][1]] - candidates[row][col]
                                if (c_cell1 == c_cell2) and (candidates[comb[0][0],comb[0][1]] != candidates[comb[1][0],comb[1][1]]):
                                    
                                    #Get all the cells seen by both locations in this comb
                                    #Remove c_cell1/2 from all these cells
                                    seen_cells1 = set([(comb[0][0],i) for i in range(9)] + [(i,comb[0][1]) for i in range(9)] + [(3*(comb[0][0]//3)+i,3*(comb[0][1]//3)+j) for i in range(3) for j in range(3)])
                                    seen_cells2 = set([(comb[1][0],i) for i in range(9)] + [(i,comb[1][1]) for i in range(9)] + [(3*(comb[1][0]//3)+i,3*(comb[1][1]//3)+j) for i in range(3) for j in range(3)])
                                    common_cells = seen_cells1 & seen_cells2
                                    #print("Y-wing removing {0} from {1} based on start corner {2} and wings {3}".format(c_cell1,common_cells,(row,col),comb))
                                    for common_loc in common_cells:
                                        if c_cell1 & candidates[common_loc[0],common_loc[1]]:
                                            candidates[common_loc[0],common_loc[1]] -= c_cell1
                                            ret=True
                                            #print("Y-wing removing {0} from {1} based on start corner {2} and wings {3}".format(c_cell1,common_loc,(row,col),comb))
    return ret

In [12]:
def xyz_wing(grid,candidates):
    #Effectively the same as a y-wing, but the start "hinge" is a three candidate cell
    #Total candidates among the three cells are still three, but hinge has all three
    #Other two have only two candidates and all three contain a common digit
    ret = False
    for row in range(9):
        for col in range(9):
            if len(candidates[row][col]) == 3:
                #Possible wing hinge cell
                #Get all the cells that are seen by this cell
                #print("Start")
                #print(row,col)
                possible_cells = []
                for col2 in range(9):
                    if col != col2 and len(candidates[row][col2]) == 2:
                        possible_cells.append((row,col2))
                for row2 in range(9):
                    if row != row2 and len(candidates[row2][col]) == 2:
                        possible_cells.append((row2,col))
                for i in range(3*(row//3),3*(row//3)+3):
                    for j in range(3*(col//3),3*(col//3)+3):
                        if (i,j) != (row,col):
                            if len(candidates[i][j]) == 2:
                                possible_cells.append((i,j))
                #print(possible_cells)
                possible_cells = list(set(possible_cells)) #Just eliminates duplicates that occur from box search
                #print(possible_cells)
                #print("")
                if len(possible_cells) >= 2:
                    #Then we have at least the number of two candidate cells to enable an xyz-wing
                    #Splice through all two cell subsets of list
                    for comb in itertools.combinations(possible_cells,2):
                        #Comb should be tuple of two location tuples
                        
                        #Check if same region
                        box_cells = [(3*(row//3)+i,3*(col//3)+j) for i in range(3) for j in range(3)]
                        if (comb[0][0] == comb[1][0] == row) or (comb[0][1] == comb[1][1] == col) or (comb[0] in box_cells and comb[1] in box_cells): #(row,col) obviously in the same box
                            #print("Same region")
                            continue
                        else:
                            triple_check = set([])
                            for loc in comb:
                                triple_check = triple_check | candidates[loc[0],loc[1]]
                            triple_check = triple_check | candidates[row][col]
                            if len(triple_check) == 3:
                                #print(row,col)
                                #print(comb)
                                #print(grid)
                                #print(candidates)

                                #Ok now we gotta check if we have ab, bc, ac condition met
                                if (candidates[comb[0][0],comb[0][1]] != candidates[comb[1][0],comb[1][1]]) and len(candidates[comb[0][0],comb[0][1]] & candidates[comb[1][0],comb[1][1]]) == 1:
                                    #Then the two others are non-equal but have one digit in common
                                    #Therefore the hinge has xyz, one cell has xz and other has yz
                                    common = candidates[comb[0][0],comb[0][1]] & candidates[comb[1][0],comb[1][1]]
                                    
                                    #Get all the cells seen by both locations in this comb
                                    #Remove c_cell1/2 from all these cells
                                    seen_cells1 = set([(comb[0][0],i) for i in range(9)] + [(i,comb[0][1]) for i in range(9)] + [(3*(comb[0][0]//3)+i,3*(comb[0][1]//3)+j) for i in range(3) for j in range(3)])
                                    seen_cells2 = set([(comb[1][0],i) for i in range(9)] + [(i,comb[1][1]) for i in range(9)] + [(3*(comb[1][0]//3)+i,3*(comb[1][1]//3)+j) for i in range(3) for j in range(3)])
                                    seen_cells3 = set([(row,i) for i in range(9)] + [(i,col) for i in range(9)] + [(3*(row//3)+i,3*(col//3)+j) for i in range(3) for j in range(3)])
                                    common_cells = seen_cells1 & seen_cells2 & seen_cells3
                                    #print("Y-wing removing {0} from {1} based on start corner {2} and wings {3}".format(c_cell1,common_cells,(row,col),comb))
                                    for common_loc in common_cells:
                                        if common & candidates[common_loc[0],common_loc[1]]:
                                            candidates[common_loc[0],common_loc[1]] -= common
                                            ret=True
                                            #print("XYZ-wing removing {0} from {1} based on hinge {2} and wings {3}".format(common,common_loc,(row,col),comb))
    return ret

In [13]:
def is_solution(grid):
    if np.count_nonzero(grid) < len(grid[0])**2:
        return False
    return True

In [14]:
def is_valid_grid(grid,candidates):
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0 and len(candidates[i][j]) == 0:
                #No fill,
                return False
    return True

In [15]:
def validKnightMoves(knight_positions = []):
    #Returns for each knight placed on the field, the knight position and any valid knight move coordinates
    knight_move_positions = []
    for knight_start in knight_positions:
        valid_set = [knight_start] + [(knight_start[0]+moveset[0],knight_start[1]+moveset[1]) for moveset in [(2,1),(1,2),(-1,2),(-2,1),(-2,-1),(-1,-2),(1,-2),(2,-1)] if (((knight_start[0]+moveset[0]) >= 0) and ((knight_start[0]+moveset[0]) < 9) and ((knight_start[1]+moveset[1]) >= 0) and ((knight_start[1]+moveset[1]) < 9))]
        knight_move_positions.append(valid_set)
    return knight_move_positions

In [16]:
def is_valid_knight_sums(grid,knights):
    if knights == []:
        return True
    knight_sum_position_groups = validKnightMoves(knights)
    #If the grid values for any two or more sets of knight sum positions are all non-zero
    #Then we can determine if grid is invalid from unequal sums
    knight_grid_value_groups = []
    for position_group in knight_sum_position_groups:
        lst = []
        for position in position_group:
            lst.append(grid[position[0]][position[1]])
        knight_grid_value_groups.append(lst)
    non_zero_index = [i for i in range(len(knight_grid_value_groups)) if np.all(knight_grid_value_groups[i])]
    if len(knights) > 1: #If we have more than one knight, then must prove that all two or more fully filled knight positions are the same sum
        if len(non_zero_index) > 1: #If we have more than one filled in knight sum, confirm all = sums
            sums = []
            masked = list(np.array(knight_grid_value_groups)[non_zero_index])
            for values in masked:
                s = 0
                for val in values:
                    s+= val
                sums.append(s)
            if len(set(sums)) != 1:
                #Gah! Unequal sums!
                return False
    return True

In [17]:
def guess(grid,candidates,knights,knight_constraint):
    #Get location to guess
    indices = np.arange(81)
    vals = np.array([a for a in map(len, np.array(candidates).reshape(1,81)[0])])
    vals,indices = zip(*sorted(zip(vals,indices),reverse=False))
    vals = np.array(vals)
    indices = np.array(indices)
    indices = indices[vals!=0]
    #vals = vals[vals!=0]
    index_choice = indices[0]
    row = index_choice // 9
    col = index_choice % 9
    #print(grid)
    #print(candidates)
    #Solution must be one of these candidates so try them all
    '''
    print("Top of guess")
    print(grid)
    print(candidates)
    '''
    cell_candidates = candidates[row][col]
    #print("Testing cell row {0} col {1} with candidates {2}".format(row,col,cell_candidates))
    grid_copy = grid.copy()
    candidates_copy = copy.deepcopy(candidates)
    #print(candidates_copy)
    for guess in candidates_copy[row][col]:
        #Try a guess
        #print(row,col,guess)
        grid[row][col] = guess
        candidates[row][col] = set([])
        adjusted_cells = reduce_candidates(candidates,row,col,guess)
        '''
        print("Candidates after placing value {0} at {1},{2}".format(guess,row,col))
        print(grid)
        print(candidates)
        '''
        #Proceed to attempt to solve the puzzle normally
        solution = solve(grid,candidates,knights,knight_constraint)
        if solution is not None:
            return solution
        #Undo the guess
        candidates = copy.deepcopy(candidates_copy)
        grid = grid_copy.copy()
        '''
        print("Resetting")
        print(candidates)
        print(candidates_copy)
        print(grid)
        '''

In [18]:
def knight_constraint_deductions(grid,candidates):
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                #Digit placed there
                knight_moves = validKnightMoves([(i,j)])[0]
                for k in range(1,len(knight_moves)):
                    #This way it ignores the actual knight position returned by the knight move function
                    candidates[knight_moves[k][0]][knight_moves[k][1]] -= set([grid[i][j]])
                    if grid[i][j] == grid[knight_moves[k][0]][knight_moves[k][1]]:
                        #That's not good
                        #print(grid)
                        #print("Grid ({0},{1}) = Grid ({2},{3})".format(i,j,knight_moves[k][0],knight_moves[k][1]))
                        return False
    return True

In [19]:
def solve(grid,candidates,knights = [],knight_constraint=False):
    solved = False
    while not solved:
        #Step through most basic deductions to hardest, then guess recursively
        if knight_constraint:
            if not knight_constraint_deductions(grid,candidates):
                return None
        if naked_singles(grid,candidates):
            '''
            print("Naked Singles")
            print(grid)
            print(candidates)
            '''
            continue
        elif hidden_singles_up(grid,candidates):
            '''
            print("Hidden Groups")
            print(grid)
            print(candidates)
            '''
            continue
        elif naked_pair_up(grid,candidates):
            '''
            print("Naked Group")
            print(grid)
            print(candidates)
            '''
            continue
        elif pointing_pairs(grid,candidates):
            '''
            print("Pointing Pairs")
            print(grid)
            print(candidates)
            '''
            continue
        elif box_line_reduction(grid,candidates):
            '''
            print("Box Reduction")
            print(grid)
            print(candidates)
            '''
            continue
        elif x_wing(grid,candidates):
            '''
            print("x wing")
            print(grid)
            print(candidates)
            '''
            continue
        elif y_wing(grid,candidates):
            continue
        elif xyz_wing(grid,candidates):
            continue
        elif is_solution(grid):
            solved = True
            return grid,candidates
        #else:
        #    return grid,candidates
        #'''
        else:
            print("Guessing")
            #None of the standard approaches worked
            #First check if we have solved the grid
            #print(grid)
            #print("---------------------------------------------------------------------------------------------------------------------------------------")
            if not is_valid_grid(grid,candidates):
                #Apparently there is an unfilled cell with no candidates, this was a bad grid
                #Return None back to guess to have it know to try a new guess
                #'''
                print("Invalid")
                #print(grid)
                #print(candidates)
                #'''
                return None
            if not is_valid_knight_sums(grid,knights):
                print("Knight sum invalid")
                return None
            if is_solution(grid):
                #Grid is filled, assumed correctly, and returned
                solved = True
                return grid,candidates
            #If neither then try a guess!
            #print("guessing")
            return guess(grid,candidates,knights,knight_constraint)
        #'''
    

In [20]:
puzzle = '''000000000
            600000000
            000001094
            900004070
            000608000
            010020003
            820500000
            000000005
            034090710
         '''
puzzle = '''900240000
            050690231
            020050090
            090700320
            002935607
            070002900
            069020073
            510079062
            207086009
         '''
puzzle = '''600000008
            500908007
            820001030
            340209080
            200080300
            180307025
            750400092
            900005004
            400090003
         '''
#Swordfish puzzle elim on columns
puzzle = '''020040069
            003806200
            060020000
            890500010
            000000000
            030001026
            000010070
            009604300
            270050090
         '''
#Swordfish puzzle elim on columns
puzzle = '''900000000
            037010420
            840000603
            000034810
            000060000
            068120000
            102000084
            085070360
            000000001
         '''
#knights = [(0,1),(1,0),(6,6),(6,7),(7,6),(8,0)]
grid = create_grid(puzzle)
candidates = create_candidates(grid)
#grid,candidates
'''

candidates = np.array([[set([1,2,3,4]),set(np.arange(1,10)),set(np.arange(1,10)),set([1,2,3]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set([1,2,3,4,5]),set([1,2,3,4,5]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set([1,2,3,4,5]),set([7,8]),set([1,2,3]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set([1,2,3]),set(np.arange(1,10)),set([1,2,3]),set([3,4]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([8,9]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([1,2]),set([8,9]),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([8,9]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set(np.arange(1,10)),set([]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([7,8]),set(np.arange(1,10))],
                        [set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([1,2])]])
'''

'\n\ncandidates = np.array([[set([1,2,3,4]),set(np.arange(1,10)),set(np.arange(1,10)),set([1,2,3]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],\n                        [set(np.arange(1,10)),set([1,2,3,4,5]),set([1,2,3,4,5]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],\n                        [set(np.arange(1,10)),set([1,2,3,4,5]),set([7,8]),set([1,2,3]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],\n                        [set([1,2,3]),set(np.arange(1,10)),set([1,2,3]),set([3,4]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10))],\n                        [set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set([8,9]),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1,10)),set(np.arange(1

In [21]:
s = time.time()
solution = solve(grid,candidates)
if solution is None:
    print("No solution")
else:
    print(solution[0]) #candidates = solution[1]
#print(grid)
#print(candidates)
e = time.time()
print("Runtime",e-s)

[[9 2 6 5 4 3 1 7 8]
 [5 3 7 6 1 8 4 2 9]
 [8 4 1 2 9 7 6 5 3]
 [2 5 9 7 3 4 8 1 6]
 [7 1 4 8 6 9 2 3 5]
 [3 6 8 1 2 5 9 4 7]
 [1 9 2 3 5 6 7 8 4]
 [4 8 5 9 7 1 3 6 2]
 [6 7 3 4 8 2 5 9 1]]
Runtime 0.0366518497467041


C:\Users\35645\Anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


# Testing The Upper Left 4x4 Region for my sudoku puzzle

In [22]:
def smoltest(grid):
    for row in range(4):
        for col in range(4):
            if not grid[row][col]:
                for num in range(1,10):
                    grid[row][col] = num
                    if not smoltest(grid):
                        grid[row][col] = 0
            else:
                #Then theres a digit here already
                continue
    if is_smol_valid(grid):
        #Additionally validate that the two sum regions are equivalent
        if grid[0][3] == 2:
            print(grid)
        if is_smol_sum_valid(grid):
            print(grid)
            return True
    return False

In [23]:
def is_smol_sum_valid(grid):
    #This is explicitly the knight sum set by placing a knight in r0c1 and r1c0
    sum_loc = [[(0,1),(2,0),(2,2),(1,3)],[(1,0),(2,2),(0,2),(3,1)]]
    sums = [0,0]
    for i in range(2):
        for loc in sum_loc[i]:
            sums[i] += grid[loc[0],loc[1]]
    if sums[0] == sums[1]:
        print(sums[0])
        return True
    return False

In [24]:
def is_smol_valid(grid):
    #Just need no duplicates in any row, column, or box region
    for row in range(4):
        if len(set(grid[row])) != 4:
            return False
    for col in range(4):
        if len(set(grid[:,col])) != 4:
            return False
    #If the sliver making box two is also confined entirely in the column 4 constraint
    #Then those upper three digits being unique should already pass if the column is unique
    #Same for row 4, so I think only box 1 needs checked since box 4 is 1 cell
    digits = set([grid[i][j] for i in range(3) for j in range(3)])
    if len(digits) != 9:
        return False
    return True